In [2]:
import json
import pandas

In [14]:
class Download(object):
    
    def __init__(self, incoming_file, outgoing_file, packages_file):
        self.incoming_records = self._file_as_dict_records(incoming_file)
        self.outgoing_records = self._file_as_dict_records(outgoing_file)
        self.packages_records = self._file_as_dict_records(packages_file)
    
    def _file_as_dict_records(self, filepath):
        df = pandas.read_excel(filepath, converters={'package_id':str})
        print('Opening file {} with columns {}'.format(filepath, df.columns))
        return df.to_dict('records')
    
d = Download(
    incoming_file='data/royal/royal_apothecary_incoming_transfer_packages_20210101_20210905.xlsx',
    outgoing_file='data/royal/royal_apothecary_outgoing_transfer_packages_20210101_20210905.xlsx',
    packages_file='data/royal/royal_apothecary_active_inventory_20210906.xlsx',
)

/Users/davidlluncor/GitHub/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Opening file data/royal/royal_apothecary_incoming_transfer_packages_20210101_20210905.xlsx with columns Index(['date_type', 'transfer_row_id', 'delivery_row_id', 'package_row_id',
       'delivery_type', 'manifest_number', 'created_date', 'received_datetime',
       'shipper_facility_license_number', 'shipper_facility_name',
       'recipient_facility_license_number', 'recipient_facility_name',
       'shipment_type_name', 'shipment_transaction_type', 'package_id',
       'package_label', 'type', 'product_category_name', 'product_name',
       'shipper_wholesale_price', 'shipped_quantity', 'lab_results_status'],
      dtype='object')
Opening file data/royal/royal_apothecary_outgoing_transfer_packages_20210101_20210905.xlsx with columns Index(['date_type', 'transfer_row_id', 'delivery_row_id', 'package_row_id',
       'delivery_type', 'manifest_number', 'created_date',
       'shipper_facility_license_number', 'shipper_facility_name',
       'recipient_facility_license_number', 'recipie

/Users/davidlluncor/GitHub/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/davidlluncor/GitHub/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Opening file data/royal/royal_apothecary_active_inventory_20210906.xlsx with columns Index(['identifier', 'license_number', 'last_modified_at', 'package_id',
       'package_label', 'type', 'packaged_date', 'package_type',
       'product_name', 'product_category_name', 'quantity', 'unit_of_measure'],
      dtype='object')


In [19]:
from typing import Dict, List, Tuple

class PackageHistory():
    
    def __init__(self):
        self.incomings = []
        self.outgoings = []
        self.pkg = None
        
    
def get_histories(d: Download) -> Dict[str, PackageHistory]:
    package_id_to_history = {}
    
    for in_r in d.incoming_records:
        package_id = in_r['package_id']
        if package_id not in package_id_to_history:
            package_id_to_history[package_id] = PackageHistory()
            
        history = package_id_to_history[package_id]
        history.incomings.append(in_r)

    for out_r in d.outgoing_records:
        package_id = out_r['package_id']
        if package_id not in package_id_to_history:
            package_id_to_history[package_id] = PackageHistory()
            
        history = package_id_to_history[package_id]
        history.outgoings.append(out_r)
        
    
    for pkg_r in d.packages_records:
        package_id = pkg_r['package_id']
        if package_id not in package_id_to_history:
            package_id_to_history[package_id] = PackageHistory()
            
        history = package_id_to_history[package_id]
        history.pkg = pkg_r
        
    return package_id_to_history
    
id_to_history = get_histories(d)

In [29]:
def print_counts(id_to_history):
    only_incoming = 0
    only_outgoing = 0
    outgoing_and_incoming = 0
    current_inventory = 0
    inventory_with_no_transfers = 0
    total_seen = 0

    for package_id, history in id_to_history.items():
        if history.outgoings and not history.incomings:
            only_outgoing += 1

        if history.incomings and not history.outgoings:
            only_incoming += 1

        if history.pkg:
            current_inventory += 1
            
        if history.outgoings and history.incomings:
            outgoing_and_incoming += 1
            
        if history.pkg and not history.outgoings and not history.incomings:
            inventory_with_no_transfers += 1

        total_seen += 1

    print(f'Only outgoing: {only_outgoing}')
    print(f'Only incoming: {only_incoming}')
    print(f'In and out: {outgoing_and_incoming}')
    print(f'Inventory no transfers: {inventory_with_no_transfers}')
    print(f'Cur inventory: {current_inventory}')
    print(f'Total pkgs: {total_seen}')
    
print_counts(id_to_history)

Only outgoing: 33
Only incoming: 2255
In and out: 1
Inventory no transfers: 908
Cur inventory: 1471
Total pkgs: 3197


In [ ]:
import time

xlsx_file_name = f'~/Downloads/export_data_{round(time.time())}.xlsx'
raw_incoming_transfers_dataframe.to_excel(xlsx_file_name, index=False)